In [3]:
import winsound

def beepStart():
    d = 250
    freq = [2500,3000,3500,4000]
    for i in freq:
        winsound.Beep(i, d)
        
def beepLoop():
    d = 100
    freq = [1000,2000]
    for i in freq:
        winsound.Beep(i, d)
        
def beepMid():
    d = 200
    freq = [3000,3000,3000]
    for i in freq:
        winsound.Beep(i, d)     
        
def beepLoopMid():
    d = 500
    freq = [1400,1800]
    for i in freq:
        winsound.Beep(i, d)
        
def beepEnd():
    d = 200
    freq = [3000,5000,3000,5000]
    for i in freq:
        winsound.Beep(i, d)

In [4]:
import warnings
import pandas as pd 
import numpy as np
import pickle 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from itertools import combinations
from itertools import permutations


def acceptanceProbability (bestMSE, newMSE, temp , K):
        # If the new solution is worse, calculate an acceptance probability
        return np.exp( K * (newMSE - bestMSE) / temp)
    
def calcMSE(combDesc, x_train, x_val, y_train, y_val, model):
    x_train_slice = x_train.iloc[:,combDesc]
    x_val_slice = x_val.iloc[:,combDesc]
    model.fit(x_train_slice, y_train)
    y_pred = model.predict(x_val_slice)
    return mean_squared_error(y_val, y_pred)

warnings.filterwarnings('ignore')
unpick_dtTrain = pd.read_pickle("./dtTrain.pkl")
inDt = open("dtTrain.pkl","rb")
dtTrain = pickle.load(inDt)
# dtTrain.head()

In [5]:
descName = unpick_dtTrain.columns.to_list()
# descName

In [6]:
##### SIMULATED ANNEALING #####

# 0. Preparation Data
train, val = train_test_split(dtTrain, test_size = 0.2, random_state = 45)

x_train = train.iloc[:,:-1]
x_val = val.iloc[:,:-1]
y_train = train.iloc[:, [-1]]
y_val = val.iloc[:,[-1]]

model = LinearRegression()

In [7]:
##### INTO DE SA DE LA'SOIN #####
descNum = 6
descMSE_20 = []
midSound = True

beepStart()
for j in range(20):
    
    #Initiate Values
    initTemp, temp = 100, 100
    tempEnd = 0.1
    cooling_rate = 0.98
    iteration = 5
    
    ### 3.3 Simulated Anyeong
    bestList = []
    bestMSEList = []
    sounded = True
    
    #3.1 Initialize solution & Best MSE (Random 6 descriptor)
    bestSol = np.random.choice(x_train.shape[1],descNum, replace=False)
    bestSol = list(bestSol)
    bestSol.sort()
    print(j+1,".Init Desc : ", bestSol)

    # MSE
    bestMSE = calcMSE(bestSol, x_train, x_val, y_train, y_val, model)
    print("Init MSE : ", bestMSE)
    beepLoop()
    while temp > tempEnd:
        for i in range(iteration):

            #Create new solution & new MSE
            newSol = np.random.choice(x_train.shape[1], descNum, replace=False)
            newMSE = calcMSE(newSol, x_train, x_val, y_train, y_val, model)
            deltaMSE = newMSE - bestMSE
            K = (initTemp * np.log(0.8)) / deltaMSE

            if newMSE < bestMSE :
                bestMSE = newMSE
                bestSol = newSol
                bestList.append([bestMSE,bestSol])
            else :
                if acceptanceProbability (bestMSE, newMSE, temp , K) > np.random.rand(0,1):
                    bestMSE = newMSE
                    bestSol = newSol
                    bestList.append([bestMSE,bestSol])

        temp *= cooling_rate
        bestMSEList.append(bestMSE)
    
    if(j > 4 and midSound):
        midSound = False
        beepLoopMid()
    
    descMSE_20.append([bestList[-1][0],bestList[-1][1],bestMSEList])
    print('Loop',j+1,'done!')
    
beepEnd()
print(" ===== Loops Finish ===== ")

1 .Init Desc :  [1, 8, 14, 29, 61, 62]
Init MSE :  0.47735691543680153
Loop 1 done!
2 .Init Desc :  [9, 16, 30, 41, 47, 60]
Init MSE :  0.9788966172210201
Loop 2 done!
3 .Init Desc :  [1, 12, 18, 23, 43, 51]
Init MSE :  0.34622550123908324
Loop 3 done!
4 .Init Desc :  [6, 8, 9, 34, 39, 71]
Init MSE :  1.1422673703766377
Loop 4 done!
5 .Init Desc :  [6, 19, 48, 65, 67, 71]
Init MSE :  0.8088112684196808
Loop 5 done!
6 .Init Desc :  [5, 23, 24, 31, 48, 68]
Init MSE :  0.5395192589408111
Loop 6 done!
7 .Init Desc :  [16, 29, 40, 48, 54, 60]
Init MSE :  0.6499228481812501
Loop 7 done!
8 .Init Desc :  [4, 17, 35, 37, 38, 49]
Init MSE :  0.720690561692526
Loop 8 done!
9 .Init Desc :  [5, 20, 28, 43, 60, 68]
Init MSE :  0.5436544118498341
Loop 9 done!
10 .Init Desc :  [15, 23, 36, 40, 65, 69]
Init MSE :  0.6984927336146548
Loop 10 done!
11 .Init Desc :  [24, 45, 46, 50, 59, 60]
Init MSE :  0.5316487820804876
Loop 11 done!
12 .Init Desc :  [4, 10, 26, 47, 59, 70]
Init MSE :  0.8287256211125557

In [15]:
pd.DataFrame(descMSE_20)

,0,1,2
0,0.268747,"[18, 55, 40, 69, 28, 11]","[0.401877621104587, 0.39883966465746706, 0.398..."
1,0.256768,"[32, 40, 36, 51, 11, 70]","[0.5373651946585759, 0.29614395914625846, 0.29..."
2,0.255658,"[52, 20, 36, 18, 58, 61]","[0.34622550123908324, 0.34622550123908324, 0.3..."
3,0.279011,"[65, 18, 36, 69, 57, 33]","[0.572148103954527, 0.5362678554381664, 0.5362..."
4,0.328399,"[71, 11, 3, 60, 69, 64]","[0.5275593936274945, 0.5275593936274945, 0.386..."
5,0.325893,"[65, 18, 51, 1, 64, 23]","[0.5013672963233549, 0.4644031540767506, 0.464..."
6,0.216492,"[55, 18, 51, 70, 40, 63]","[0.4498720682801387, 0.4498720682801387, 0.396..."
7,0.290282,"[69, 40, 16, 17, 11, 26]","[0.5706418305846561, 0.5706418305846561, 0.419..."
8,0.276320,"[5, 24, 55, 45, 15, 51]","[0.5436544118498341, 0.5436544118498341, 0.543..."
9,0.293327,"[69, 17, 38, 27, 8, 68]","[0.3120660419773643, 0.3120660419773643, 0.312..."


In [16]:
dfBestDM = pd.DataFrame(descMSE_20)
dfBestDM.columns=["MSE","solution","growth"]
dfBestDM.sort_values('MSE', inplace=True)
dfBestDM.reset_index(drop=True, inplace= True)
dfBestDM.iloc[0,2]

[0.4498720682801387,
 0.4498720682801387,
 0.3966520367978579,
 0.3966520367978579,
 0.3966520367978579,
 0.3966520367978579,
 0.3966520367978579,
 0.3966520367978579,
 0.3966520367978579,
 0.3966520367978579,
 0.3966520367978579,
 0.3966520367978579,
 0.3966520367978579,
 0.3966520367978579,
 0.3966520367978579,
 0.3966520367978579,
 0.3966520367978579,
 0.3854652101416471,
 0.3854652101416471,
 0.3854652101416471,
 0.3854652101416471,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027

In [20]:
bestDescriptor = []
for i in dfBestDM.iloc[0,1]:
    bestDescriptor.append(descName[i])

bestMSEGrowth = dfBestDM.iloc[0,2]

In [21]:
# Pickle descriptor name
pickle.dump(bestDescriptor, open('./data/bestDescV1.pkl', 'wb'))
pickle.dump(bestMSEGrowth, open('./data/bestMSEGrowthV1.pkl','wb'))

In [ ]:
############## CUSTOMIZED CODE ##################################

In [ ]:
# bestMSEList
dfBest.columns = ["mse","solution"]
dfBest.sort_values('mse', inplace=True)
dfBest.reset_index(drop=True, inplace= True)
dfBest

In [ ]:
bestSolution = dfBest.loc[0,'solution']
bestSolution = list(bestSolution)
bestSolution.sort()
bestSolution

In [ ]:
bestDescriptor = []

for i in bestSolution:
    bestDescriptor.append(descName[i])
bestDescriptor

In [ ]:
mseNdarray = np.asarray(bestMSEList)
type(mseNdarray)
mseNdarray.dump("./bestMSEList_2.pkl")
dfBest.iloc[-1][1].dump("./bestDesc_2.pkl")

In [ ]:
len(mseNdarray)
np.arange(0,len(mseNdarray),1)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

#define variable x
x = np.arange(0,len(mseNdarray),1)
y = mseNdarray 

#plot the functions
plt.grid()
plt.xlabel("Iteration")
plt.ylabel("Mean Squared Error (MSE)")
plt.plot(x,y)

In [ ]:
### Smallest Values before run 20s
bs = [38, 51, 55, 58, 36, 18]
bs.sort()
bs

In [ ]:
bds = []
for i in bs:
    bds.append(descName[i])
bds

In [ ]:
pickle.dump(bds,open('./BestDescriptor.pkl','wb'))

In [22]:
bestDesc = pickle.load(open('./data/bestDescV1.pkl', 'rb'))
bestMSEGrowth = pickle.load(open('./data/bestMSEGrowthV1.pkl', 'rb'))

In [24]:
bestDesc

['VR1_D.1', 'ATSC8i', 'minHBint8', 'RNCS.1', 'AATSC6v.1', 'TDB10i']

In [25]:
bestMSEGrowth

[0.4498720682801387,
 0.4498720682801387,
 0.3966520367978579,
 0.3966520367978579,
 0.3966520367978579,
 0.3966520367978579,
 0.3966520367978579,
 0.3966520367978579,
 0.3966520367978579,
 0.3966520367978579,
 0.3966520367978579,
 0.3966520367978579,
 0.3966520367978579,
 0.3966520367978579,
 0.3966520367978579,
 0.3966520367978579,
 0.3966520367978579,
 0.3854652101416471,
 0.3854652101416471,
 0.3854652101416471,
 0.3854652101416471,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027147648,
 0.3513162027

In [ ]:
# First Best Run
# BestDescriptors=  [31,18,54,36,8,51]
# BestMSE =  0.24519892533514573

# Second Best Run
# Best Descriptors :  [53 61 18 69 36 34]
# Best MSE :  0.22850883728849938

# Best Descriptors :  [40 11  5 67 64 69]
# Best MSE :  0.22858482530945248

# Best Descriptors :  [38 51 55 58 36 18]
# Best MSE :  0.20720460921006825